In [ ]:
%%capture
# First install necessary libraries if not already installed
!pip install pulp gspread oauth2client

In [ ]:
# Import required libraries
import gspread
import random
from google.colab import auth
from google.auth import default
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, value

import pandas as pd


In [ ]:
def read_preferences_from_spreadsheet(spreadsheet_url, tab_name):
    """
    Read student project preferences from a Google Spreadsheet using Colab authentication.

    Parameters:
    - spreadsheet_url: URL of the Google spreadsheet
    - tab_name: Name of the tab containing preference data

    Returns:
    - students: List of student IDs
    - projects: List of project IDs
    - preferences: Dictionary mapping students to their project preferences with scores
    """

    # Authenticate with user's Google account in Colab
    auth.authenticate_user()
    # Get credentials
    creds, _ = default()
    # Create gspread client
    gc = gspread.authorize(creds)

    # Open your spreadsheet
    spreadsheet = gc.open_by_url(spreadsheet_url).worksheet(tab_name)

    # Get all values
    data = spreadsheet.get_all_values()
    preferences_df = pd.DataFrame(data[1:], columns=data[0]).set_index('').drop('Partner', axis=1)

    students = preferences_df.index.tolist()
    projects = preferences_df.columns.tolist()

    # Get header row
    header = data[0]
    rows = data[1:]

    if not students:
        print("Error: No valid student IDs found in the first column")
        return None, None, None

    print(f"Found {len(students)} students")

    if not projects:
        print("Error: No valid project IDs found in columns 2-6")
        return None, None, None

    print(f"Found {len(projects)} unique projects: {projects}")

    # Create preferences dictionary with scores (5 for 1st choice, 4 for 2nd, etc.)

    preferences_dict = preferences_df.to_dict(orient='index')

    preferences_dict = {
        student_name: sorted(
            [(company_name, 6 - int(rank)) for company_name, rank in preferences.items() if rank],
            key=lambda preference: preference[1],
            reverse=True
        )
        for student_name, preferences in preferences_dict.items()
    }

    print(f"Successfully loaded preferences for {len(preferences_dict)} students")

    return students, projects, preferences_dict

# Function to solve the assignment problem
def solve_project_assignment(students, projects, preferences, max_quota=3, project_weight=1, random_seed=None):
    """
    Solve the student-project assignment problem using integer programming.

    Parameters:
    - students: List of student IDs
    - projects: List of project IDs
    - preferences: Dictionary mapping students to their project preferences with scores
    - max_quota: Maximum students per project (default 3)
    - project_weight: Weight for the project utilization objective (default 0.1)
                     Higher values prioritize using more projects

    Project constraints:
    - Each project must have either 0, 2, or 3 students (not 1 or more than 3)

    Objectives (in order of priority):
    1. Maximize student preference satisfaction
    2. Maximize the number of projects used

    Returns:
    - Dictionary with the assignment results and metrics
    """
    random.seed(None)
    random.shuffle(students)
    # Create optimization problem
    model = LpProblem("Student-Project-Allocation", LpMaximize)

    # Decision variables for student-project assignments
    x = { (i, j): LpVariable(f"x_{i}_{j}", cat="Binary")
          for i in students for j_score in preferences.get(i, []) for j, _ in [j_score] }

    # Decision variables for project usage
    # y[j] = 1 if project j is used (has students assigned), 0 otherwise
    y = { j: LpVariable(f"y_{j}", cat="Binary") for j in projects }

    # Decision variables for project size
    # z2[j] = 1 if project j has exactly 2 students, 0 otherwise
    # z3[j] = 1 if project j has exactly 3 students, 0 otherwise
    z2 = { j: LpVariable(f"z2_{j}", cat="Binary") for j in projects }
    z3 = { j: LpVariable(f"z3_{j}", cat="Binary") for j in projects }

    # Objective function:
    # 1. Primary objective: maximize preference score
    # 2. Secondary objective: maximize number of projects used (with a lower weight)
    model += (lpSum(x[i, j] * score for i in students
                  for j_score in preferences.get(i, [])
                  for j, score in [j_score]) +
             project_weight * lpSum(y[j] for j in projects))

    # Constraint: Each student is assigned to exactly one project
    for i in students:
        if i in preferences and preferences[i]:  # Only add constraint if student has preferences
            model += lpSum(x[i, j] for j_score in preferences[i] for j, _ in [j_score]) == 1

    # Constraints for project usage and size
    for j in projects:
        # Find all students who have this project in their preferences
        students_with_this_project = [
            i for i in students
            if i in preferences and any(pref[0] == j for pref in preferences[i])
        ]

        # Skip constraints for projects that don't appear in any preferences
        if not students_with_this_project:
            continue

        # Count of students assigned to project j
        project_count = lpSum(x[i, j] for i in students_with_this_project)

        # Link y[j] with whether project j has any students
        # If project_count > 0, then y[j] must be 1
        # If project_count = 0, then y[j] must be 0
        model += project_count <= max_quota * y[j]  # Forces y[j] = 1 if any students are assigned
        model += project_count >= y[j]  # Forces y[j] = 0 if no students are assigned

        # Link z2[j] with whether project j has exactly 2 students
        # If project_count = 2, then z2[j] = 1
        # If project_count ≠ 2, then z2[j] = 0
        model += project_count >= 2 * z2[j]
        model += project_count <= 2 + max_quota * (1 - z2[j])

        # Link z3[j] with whether project j has exactly 3 students
        # If project_count = 3, then z3[j] = 1
        # If project_count ≠ 3, then z3[j] = 0
        model += project_count >= 3 * z3[j]
        model += project_count <= 3 + max_quota * (1 - z3[j])

        # The project must have either 0, 2, or 3 students
        # This means if y[j] = 1, then either z2[j] = 1 or z3[j] = 1
        model += y[j] <= z2[j] + z3[j]

    # Solve the problem
    status = model.solve()

    # Process the results
    results = {
        "status": LpStatus[status],  # Convert numeric status to string
        "assignments": {j: [] for j in projects},
        "project_counts": {},
        "student_assignments": {},
        "metrics": {}
    }

    if status == 1:  # 1 corresponds to LpStatus.Optimal
        # Collect project assignments
        for i in students:
            if i not in preferences or not preferences[i]:
                results["student_assignments"][i] = {
                    "project": "NO_PREFERENCES",
                    "preference_rank": "N/A",
                    "preference_score": 0
                }
                continue

            assigned = False
            for j_score in preferences[i]:
                j, score = j_score  # Unpack the tuple
                if (i, j) in x and value(x[i, j]) > 0.5:  # Use value() function to get variable value
                    if j not in results["assignments"]:
                        results["assignments"][j] = []
                    results["assignments"][j].append(i)

                    # Find rank in original preferences
                    rank = next((idx+1 for idx, (proj, _) in enumerate(preferences[i]) if proj == j), None)

                    results["student_assignments"][i] = {
                        "project": j,
                        "preference_rank": rank,
                        "preference_score": score
                    }
                    assigned = True
                    break

            if not assigned:
                results["student_assignments"][i] = {
                    "project": "UNASSIGNED",
                    "preference_rank": "N/A",
                    "preference_score": 0
                }

        # Calculate project counts
        for j in projects:
            results["project_counts"][j] = len(results["assignments"].get(j, []))

        # Calculate metrics
        preference_satisfaction = sum(results["student_assignments"][i]["preference_score"]
                                    for i in students if "preference_score" in results["student_assignments"][i])

        # Count active projects
        active_projects = sum(1 for j in projects if results["project_counts"].get(j, 0) > 0)

        results["metrics"]["preference_satisfaction"] = preference_satisfaction
        results["metrics"]["active_projects"] = active_projects
        results["metrics"]["average_satisfaction"] = preference_satisfaction / len(students)
        results["metrics"]["objective_value"] = value(model.objective)

        # Count preference distribution
        preference_distribution = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, "unassigned": 0, "no_preferences": 0}
        for i in students:
            assignment = results["student_assignments"][i]
            if assignment["project"] == "UNASSIGNED":
                preference_distribution["unassigned"] += 1
            elif assignment["project"] == "NO_PREFERENCES":
                preference_distribution["no_preferences"] += 1
            else:
                rank = assignment["preference_rank"]
                if isinstance(rank, int) and 1 <= rank <= 5:
                    preference_distribution[rank] += 1

        results["metrics"]["preference_distribution"] = preference_distribution

        # Add list of projects that have no students
        results["metrics"]["unused_projects"] = [
            j for j in projects if j not in results["assignments"] or len(results["assignments"][j]) == 0
        ]

        # Verify the constraints were met
        results["metrics"]["constraint_violations"] = []
        for j in projects:
            count = results["project_counts"].get(j, 0)
            if count == 1 or count > 3:
                results["metrics"]["constraint_violations"].append(
                    f"Project {j} has {count} students, but should have 0, 2, or 3 students"
                )

    else:
        print(f"No optimal solution found. Status code: {status}")
        print("This may indicate that the problem is infeasible with the current constraints.")

    return results

# Example of how to call this function:
# results = solve_project_assignment(students, projects, preferences, project_weight=0.1)

# Function to help visualize and interpret the results
def print_assignment_summary(results):
    """
    Print a summary of the assignment results in a readable format.
    """
    if results["status"] != "Optimal":
        print(f"No optimal solution found. Status: {results['status']}")
        return

    print("\n=== ASSIGNMENT SUMMARY ===\n")

    # Print project assignments
    print("PROJECT ASSIGNMENTS:")
    active_projects_count = 0
    inactive_projects = []
    for j, students_list in sorted(results["assignments"].items()):
        count = len(students_list)
        if count > 0:
            active_projects_count += 1
            print(f"{j}: {', '.join(students_list)} ({count} students)")
        else:
            inactive_projects.append(j)

    if inactive_projects:
        print("\nINACTIVE PROJECTS:")
        for j in inactive_projects:
            print(f"  - {j}")

    print(f"\nTotal active projects: {active_projects_count} out of {len(results['project_counts'])}")

    # # Print student assignments
    # print("\nSTUDENT ASSIGNMENTS:")
    # for i, info in sorted(results["student_assignments"].items()):
    #     if info["project"] == "UNASSIGNED":
    #         print(f"{i}: ⚠️ UNASSIGNED!")
    #     elif info["project"] == "NO_PREFERENCES":
    #         print(f"{i}: ⚠️ NO PREFERENCES SUBMITTED")
    #     else:
    #         print(f"{i} → {info['project']} (Choice #{info['preference_rank']}, Score: {info['preference_score']})")

    # Print preference distribution
    print("\nPREFERENCE DISTRIBUTION:")
    dist = results["metrics"]["preference_distribution"]
    print(f"1st choice: {dist[1]} students")
    print(f"2nd choice: {dist[2]} students")
    print(f"3rd choice: {dist[3]} students")
    print(f"4th choice: {dist[4]} students")
    print(f"5th choice: {dist[5]} students")
    print(f"Unassigned: {dist.get('unassigned', 0)} students")
    print(f"No preferences: {dist.get('no_preferences', 0)} students")

    # Print overall metrics
    print("\nOVERALL METRICS:")
    print(f"Total preference satisfaction: {results['metrics']['preference_satisfaction']}")
    print(f"Average satisfaction score: {results['metrics']['average_satisfaction']:.2f}")
    print(f"Active projects: {results['metrics']['active_projects']}")

    # Print constraint violations
    if results["metrics"]["constraint_violations"]:
        print("\nCONSTRAINT VIOLATIONS:")
        for violation in results["metrics"]["constraint_violations"]:
            print(f"  - {violation}")
    else:
        print("\nAll constraints satisfied ✓")

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1JSXa7XKkcRrExvbdTHu62fNC6MyRVJByenkz4bOBGM0/edit?gid=1604337559#gid=1604337559'
# tab = 'Mapping4Pini-V4'
tab = 'Mapping-reduced'

w = -1

In [ ]:
w += 1
print(w)
students, projects, preferences = read_preferences_from_spreadsheet(url, tab)
assignment_results = solve_project_assignment(students, projects, preferences, max_quota=3, project_weight=w, random_seed=None)
print_assignment_summary(assignment_results)

6
Found 29 students
Found 15 unique projects: ['AI21', 'Arborknot', 'Asterra', 'Briya', 'C8Health', 'citrusX', 'Compugene', 'Gong', 'Grain', 'Imperva1', 'Imperva2', 'Imperva3', 'Loris', 'PassportCard', 'Stellantis']
Successfully loaded preferences for 29 students

=== ASSIGNMENT SUMMARY ===

PROJECT ASSIGNMENTS:
AI21: Yaniv Gal, Amir Alikulov, Yacov Finn (3 students)
Arborknot: Yulia Nudelman, Matan Goldshtein (2 students)
Asterra: Sarit Meshesha, Carmel Sagie (2 students)
Briya: Nicole Kaptsan, Jessica Anidjar (2 students)
C8Health: Yosevin Lee, Olga Mondrus (2 students)
Compugene: Yuval Dor, Maor Blumberg (2 students)
Gong: Ori Zilberberg, Ido Gil (2 students)
Grain: Alexander Motzkin, Adi Lev (2 students)
Imperva1: Eran Lazarovich, Ruslan Shuvalov (2 students)
Imperva3: May Menachem, Amit Zulan (2 students)
Loris: Gennadii Borisov, Yehudit Mandelboim (2 students)
PassportCard: Elizaveta Polotckaia, Ophir Turetz (2 students)
Stellantis: Yonatan Goldstein, Alexandra Braginsky (2 stude

In [ ]:
assignment_df = (pd.DataFrame.from_dict(assignment_results['assignments'], orient="index", columns=[f'student_{i}' for i in range(1, 4)]).
                 rename(columns={'index': 'project'}).fillna(''))
# assignment_df.loc[assignment_df.notna().sum(axis=1).sort_values(ascending=False).index]
assignment_df

,student_1,student_2,student_3
AI21,Yosevin Lee,Yaniv Gal,Amir Alikulov
Arborknot,Yulia Nudelman,Matan Goldshtein,
Asterra,Eran Lazarovich,Sarit Meshesha,
Briya,Jessica Anidjar,Elizaveta Polotckaia,
C8Health,Yacov Finn,Olga Mondrus,
citrusX,Ruslan Shuvalov,Kaiquan Mah,
Compugene,Alexander Motzkin,Yuval Dor,
Gong,Ori Zilberberg,Ido Gil,
Grain,,,
Guesty,Nicole Kaptsan,Adi Lev,


In [ ]:
w = -1

In [ ]:
w += 1
students, projects, preferences = read_preferences_from_spreadsheet(url, tab)
assignment_results = solve_project_assignment(students, projects, preferences, max_quota=3, project_weight=w)
try:
    assignment_df = (pd.DataFrame.from_dict(assignment_results['assignments'], orient="index", columns=[f'student_{i}' for i in range(1, 4)]).
                        rename(columns={'index': 'project'}).fillna(''))
except:
    assignment_df = (pd.DataFrame.from_dict(assignment_results['assignments'], orient="index", columns=[f'student_{i}' for i in range(1, 3)]).
                        rename(columns={'index': 'project'}).fillna(''))
# assignment_df.loc[assignment_df.notna().sum(axis=1).sort_values(ascending=False).index]
assignment_df


Found 27 students
Found 19 unique projects: ['AI21', 'Arborknot', 'Asterra', 'Briya', 'C8Health', 'citrusX', 'Compugene', 'Gong', 'Grain', 'Guesty', 'Imperva1', 'Imperva2', 'Imperva3', 'Loris', 'Menora', 'Nilus', 'PassportCard', 'Stellantis', 'JewishAgency']
Successfully loaded preferences for 27 students


,student_1,student_2,student_3
AI21,Maor,Yacov,Yosevin
Arborknot,Adi,Yulia,
Asterra,Eran,Sarit,
Briya,Elizaveta,Jessica,Nicole
C8Health,Matan,Olga,
citrusX,Aviv,Kaiquan,Ruslan
Compugene,Alexander,Yuval,
Gong,Ido,Ophir,Ori
Grain,,,
Guesty,,,
